<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

##### Module 2.6: More on ChiselTest
**Prev: [Putting it all Together: An FIR Filter](2.5_exercise.ipynb)**<br>
**Next: [Generators: Parameters](3.1_parameters.ipynb)**

## Motivation
Chisel 팀은 개선된 테스트 프레임워크를 개발하고 있습니다. "ChiselTest"는 다음과 같은 개선 사항을 제공합니다.

- 단위 테스트와 시스템 통합 테스트 모두에 적합
- 구성 가능한 추상화 및 계층화를 위해 설계됨
- 사용성이 높고 단위 테스트를 쉽고 고통스럽지 않고(보일러 플레이트 및 기타 넌센스 방지) 가능한 한 유용하게 작성하도록 권장합니다.

### Planned
- 여러 백엔드 및 시뮬레이터를 대상으로 하는 기능(테스트 벡터가 정적이 아닌 경우 Scala에 대한 링크가 필요하거나 FPGA로 합성할 때 API 하위 집합을 구성하는 제한된 테스트를 사용하는 경우)
- 패키징 및 의존성 악몽을 피하기 위해 기본 chisel3에 포함됩니다.


## Setup

In [6]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: String = "/home/parkdongho/dev/chisel-bootcamp-kr/source/load-ivy.sc"

In [7]:
import chisel3._
import chisel3.util._
import chisel3.experimental._
import chisel3.experimental.BundleLiterals._
import chisel3.tester._
import chisel3.tester.RawTester.test

import chisel3._

import chisel3.util._

import chisel3.experimental._

import chisel3.experimental.BundleLiterals._

import chisel3.tester._

import chisel3.tester.RawTester.test

>이 부트캠프에는 chisel에 대해 다른 곳에서 볼 수 있는 가져오기와 약간의 차이가 필요합니다. `import chisel3.tester.RawTester.test`는 부트캠프를 위해 특별히 설계된 아래 `test(...)` 버전을 가져옵니다.

---
# Basic Tester implementation

ChiselTest는 iotester와 동일한 기본 작업으로 시작합니다. 다음은 이전 iotester와 새로운 ChiselTest 간의 기본 기능 매핑에 대한 간략한 요약입니다.

|        | iotesters             | ChiselTest            |
| :----  | :---                  | :---                |
| poke   | poke(c.io.in1, 6)     | c.io.in1.poke(6.U)    |
| peek   | peek(c.io.out1)       | c.io.out1.peek()      |
| expect | expect(c.io.out1, 6)  | c.io.out1.expect(6.U) |
| step   | step(1)               | c.io.clock.step(1)  |
| initiate | Driver.execute(...) { c => | test(...) { c => |


2.1의 간단한 통과 모듈부터 살펴보겠습니다.

In [3]:
// Chisel Code, but pass in a parameter to set widths of ports
class PassthroughGenerator(width: Int) extends Module { 
  val io = IO(new Bundle {
    val in = Input(UInt(width.W))
    val out = Output(UInt(width.W))
  })
  io.out := io.in
}

defined class PassthroughGenerator

이전 스타일을 사용하면 간단한 테스트는 다음과 같습니다.

```scala
val testResult = Driver(() => new Passthrough()) {
  c => new PeekPokeTester(c) {
    poke(c.io.in, 0)     // Set our input to value 0
    expect(c.io.out, 0)  // Assert that the output correctly has 0
    poke(c.io.in, 1)     // Set our input to value 1
    expect(c.io.out, 1)  // Assert that the output correctly has 1
    poke(c.io.in, 2)     // Set our input to value 2
    expect(c.io.out, 2)  // Assert that the output correctly has 2
  }
}
assert(testResult)   // Scala Code: if testResult == false, will throw an error
println("SUCCESS!!") // Scala Code: if we get here, our tests passed!
```



In [5]:
test(new PassthroughGenerator(16)) { c =>
    c.io.in.poke(0.U)     // Set our input to value 0
    c.io.out.expect(0.U)  // Assert that the output correctly has 0
    c.io.in.poke(1.U)     // Set our input to value 1
    c.io.out.expect(1.U)  // Assert that the output correctly has 1
    c.io.in.poke(2.U)     // Set our input to value 2
    c.io.out.expect(2.U)  // Assert that the output correctly has 2
}

Elaborating design...
Done elaborating.
test PassthroughGenerator Success: 0 tests passed in 2 cycles in 0.001687 seconds 1185.67 Hz


>ChiselTest가 CLOCK을 발전시키는 방법을 설명하기 위해 이전 예제에 몇 가지 `step` 작업을 추가할 수 있습니다.

In [13]:
test(new PassthroughGenerator(16)) { c =>
    c.io.in.poke(0.U)     // Set our input to value 0
    c.clock.step(1)    // advance the clock
    c.io.out.expect(0.U)  // Assert that the output correctly has 0
    c.io.in.poke(1.U)     // Set our input to value 1
    c.clock.step(1)    // advance the clock
    c.io.out.expect(1.U)  // Assert that the output correctly has 1
    c.io.in.poke(2.U)     // Set our input to value 2
    c.clock.step(1)    // advance the clock
    c.io.out.expect(2.U)  // Assert that the output correctly has 2
}

Elaborating design...
Done elaborating.
test PassthroughGenerator Success: 0 tests passed in 5 cycles in 0.001054 seconds 4743.26 Hz


---
## What to notice in the above example

ChiselTest의 `test` 방법은 보일러 플레이트가 조금 덜 필요합니다. `PeekPokeTester`가 이제 프로세스에 내장되었습니다.

`poke` 및 `expect` 메소드는 이제 각 개별 `io` 요소의 일부입니다. 이것은 테스터에게 더 나은 유형 검사를 위한 중요한 힌트를 제공합니다. `peek` 및 `step` 작업은 이제 `io` 요소의 메서드이기도 합니다.

또 다른 차이점은 poke되고 expect되는 값이 Chisel 리터럴이라는 것입니다. 여기에서는 매우 간단하지만 더 고급스럽고 흥미로운 예제에서 더 강력한 검사를 제공합니다. 이것은 `Bundle` 리터럴을 지정하는 기능의 향후 개선으로 더욱 향상될 것입니다.

# Modules with Decoupled Interfaces
이 섹션에서는 `Decoupled` 인터페이스로 작업하기 위한 tester2의 몇 가지 도구를 살펴보겠습니다. `Decoupled`는 Chisel 데이터 유형을 취하여 `ready` 및 `valid` 신호를 제공합니다. ChiselTest는 이러한 인터페이스를 자동화하고 안정적으로 테스트하기 위한 몇 가지 유용한 도구를 제공합니다.

## A queue example
`QueueModule`은 `ioType`에 의해 유형이 결정되는 데이터를 전달합니다. `QueueModule` 내부에는 `entries` 상태 요소가 있습니다. 이는 백프레셔를 적용하기 전에 많은 요소를 보유할 수 있음을 의미합니다.

In [14]:
class QueueModule[T <: Data](ioType: T, entries: Int) extends MultiIOModule {
  val in = IO(Flipped(Decoupled(ioType)))
  val out = IO(Decoupled(ioType))
  out <> Queue(in, entries)
}

defined class QueueModule

## EnqueueNow and expectDequeueNow

*ChiselTest*에는 IO에서 `Decoupled` 인터페이스가 있는 회로를 처리하기 위한 몇 가지 내장 메서드가 있습니다. 이 예제에서는 `queue`에서 값을 삽입하고 추출하는 방법을 볼 것입니다.

| method | description |
| :---   | :---        |
| enqueueNow | `Decoupled` 입력 인터페이스에 하나의 요소 추가(enqueue) |
| expectDequeueNow |  `Decoupled` 출력 인터페이스에서 한 요소를 제거(dequeues) |



>참고: 테스트 시작 시 `ready` 및 `valid` 필드가 모두 올바르게 초기화되도록 하는 데 필요한 일부 필수 상용구 `initSource`, `setSourceClock` 등이 있습니다.


In [15]:
test(new QueueModule(UInt(9.W), entries = 200)) { c =>
    // Example testsequence showing the use and behavior of Queue
    c.in.initSource()
    c.in.setSourceClock(c.clock)
    c.out.initSink()
    c.out.setSinkClock(c.clock)
    
    val testVector = Seq.tabulate(200){ i => i.U }

    testVector.zip(testVector).foreach { case (in, out) =>
      c.in.enqueueNow(in)
      c.out.expectDequeueNow(out)
    }
}

Elaborating design...
Done elaborating.
test QueueModule Success: 0 tests passed in 402 cycles in 0.047142 seconds 8527.49 Hz


## EnqueueSeq and DequeueSeq 
이제 단일 작업에서 대기열에 추가하고 대기열에서 빼는 작업을 처리하는 두 가지 새로운 방법을 소개합니다.

| method | description |
| :---   | :---        |
| enqueueSeq | 시퀀스가 소진될 때까지 한 번에 하나씩 `Seq`의 요소를 `Decoupled` 입력 인터페이스에 계속 추가(enqueue)합니다. |
| expectDequeueSeq | `Decoupled` 출력 인터페이스에서 요소를 한 번에 하나씩 제거(dequeue)하고 각각을 `Seq`의 다음 요소와 비교합니다. |


> 참고: 아래 예제는 잘 작동하지만 작성된 대로 `expectDequeueSeq`가 시작되기 전에 `enqueueSeq`가 완료되어야 합니다. 대기열이 가득 차서 `enqueueSeq`를 완료할 수 없기 때문에 이 예제는 `testVector`의 크기가 대기열 깊이보다 커지면 실패합니다. 실패가 어떻게 보이는지 직접 시도하십시오. 다음 섹션에서는 이러한 유형의 테스트를 올바르게 구성하는 방법을 보여줍니다.


In [16]:
test(new QueueModule(UInt(9.W), entries = 200)) { c =>
    // Example testsequence showing the use and behavior of Queue
    c.in.initSource()
    c.in.setSourceClock(c.clock)
    c.out.initSink()
    c.out.setSinkClock(c.clock)
    
    val testVector = Seq.tabulate(100){ i => i.U }

    c.in.enqueueSeq(testVector)
    c.out.expectDequeueSeq(testVector)
}

Elaborating design...
Done elaborating.
test QueueModule Success: 0 tests passed in 202 cycles in 0.023482 seconds 8602.43 Hz


> 마지막 섹션에서 한 가지 더 중요한 점은 우리가 방금 본 함수인 `enqueueNow`, `enqueueSeq`, `expectDequeueNow` 및 `expectDequeueSeq`가 ChiselTest의 복잡한 특수 케이스 논리가 아니라는 것입니다. 오히려 이들은 ChiselTest가 ChiselTest 기본 요소에서 구축하도록 권장하는 하네스 구축 유형의 예입니다. 이러한 방법이 구현되는 방법을 확인하려면 확인하십시오. [TestAdapters.scala](https://github.com/ucb-bar/chisel-testers2/blob/d199c5908828d0be5245f55fce8a872b2afb314e/src/main/scala/chisel3/tester/TestAdapters.scala)

# Fork and Join in ChiselTest

이 섹션에서는 단위 테스트의 실행 섹션을 동시에 살펴보겠습니다. 이를 위해 우리는 testers2의 두 가지 새로운 기능을 소개할 것입니다.

| method | description |
| :---   | :---        |
| fork   | 동시 코드 블록을 시작하면 선행 fork의 코드 블록 끝에 추가된 .fork를 통해 추가 fork를 동시에 실행할 수 있습니다. |
| join | 여러 관련 fork를 호출 스레드로 다시 결합합니다. |



아래 예에서는 두 개의 `fork`가 함께 연결된 다음 `join`됩니다. 첫 번째 `fork` 블록에서 `enqueueSeq`는 소진될 때까지 계속 요소를 추가합니다. 두 번째 `fork` 블록은 데이터를 사용할 수 있을 때 각 주기에서 `expectDequeueSeq`입니다.

>fork에 의해 생성된 스레드는 코드에 지정된 순서에 따라 결정적인 순서로 실행되며 다른 스레드에 의존하는 특정 버그가 발생하기 쉬운 작업은 런타임 검사에서 금지됩니다.


In [17]:
test(new QueueModule(UInt(9.W), entries = 200)) { c =>
    // Example testsequence showing the use and behavior of Queue
    c.in.initSource()
    c.in.setSourceClock(c.clock)
    c.out.initSink()
    c.out.setSinkClock(c.clock)
    
    val testVector = Seq.tabulate(300){ i => i.U }

    fork {
        c.in.enqueueSeq(testVector)
    }.fork {
        c.out.expectDequeueSeq(testVector)
    }.join()
}

Elaborating design...
Done elaborating.
test QueueModule Success: 0 tests passed in 303 cycles in 0.066343 seconds 4567.15 Hz


## Using Fork and Join with GCD
이 섹션에서는 *최대공약수* **GCD** 테스트를 구현하기 위해 분기 조인 방법을 사용할 것입니다.
IO 번들을 정의하는 것으로 시작하겠습니다. 우리는 `Bundle` *literals*를 사용할 수 있도록 여기에 약간의 상용구를 추가할 것입니다. 의견이 말했듯이 곧 우리가 리터럴 지원 코드의 자동 생성을 지원하게 되기를 바랍니다.

In [18]:
class GcdInputBundle(val w: Int) extends Bundle {
  val value1 = UInt(w.W)
  val value2 = UInt(w.W)
}

defined class GcdInputBundle

In [19]:
class GcdOutputBundle(val w: Int) extends Bundle {
  val value1 = UInt(w.W)
  val value2 = UInt(w.W)
  val gcd    = UInt(w.W)
}

defined class GcdOutputBundle

이제 **GCD**의 *Decoupled* 버전을 살펴보겠습니다. 입력 및 출력 번들에 `ready` 및 `valid` 신호를 추가하는 `Decoupled` 래퍼를 사용하도록 여기에서 약간 수정했습니다. `Flipped` 래퍼는 기본적으로 출력으로 생성되고 각 필드를 반대 방향(재귀적으로)으로 변환하는 `Decoupled` `GcdInputBundle`을 사용합니다. `Decoupled`에 번들된 인수의 데이터 요소는 최상위 필드 `bits`에 배치됩니다.

In [20]:
/**
  * Compute GCD using subtraction method.
  * Subtracts the smaller of registers x and y from the larger until register y is zero.
  * value input register x is then the Gcd
  * returns a packet of information with the two input values and their GCD
  */
class DecoupledGcd(width: Int) extends MultiIOModule {

  val input = IO(Flipped(Decoupled(new GcdInputBundle(width))))
  val output = IO(Decoupled(new GcdOutputBundle(width)))

  val xInitial    = Reg(UInt())
  val yInitial    = Reg(UInt())
  val x           = Reg(UInt())
  val y           = Reg(UInt())
  val busy        = RegInit(false.B)
  val resultValid = RegInit(false.B)

  input.ready := ! busy
  output.valid := resultValid
  output.bits := DontCare

  when(busy)  {
    // during computation keep subtracting the smaller from the larger
    when(x > y) {
      x := x - y
    }.otherwise {
      y := y - x
    }
    when(y === 0.U) {
      // when y becomes zero computation is over,
      // signal valid data to output if the output is ready
      output.bits.value1 := xInitial
      output.bits.value2 := yInitial
      output.bits.gcd := x
      output.valid := true.B
      busy := ! output.ready
    }
  }.otherwise {
    when(input.valid) {
      // valid data available and no computation in progress, grab new values and start
      val bundle = input.deq()
      x := bundle.value1
      y := bundle.value2
      xInitial := bundle.value1
      yInitial := bundle.value2
      busy := true.B
      resultValid := false.B
    }
  }
}

defined class DecoupledGcd

우리의 테스트는 이전의 Queue 테스트와 거의 동일하게 보입니다. 그러나 계산에 여러 주기가 걸리므로 각 GCD가 계산될 때 입력 대기열 프로세스가 차단되기 때문에 더 많은 일이 진행됩니다. 좋은 소식은 이것의 테스트 측면이 다른 Decoupled 회로에서 간단하고 일관성이 있다는 것입니다.

여기에 새로운 Chisel3 `Bundle` 리터럴 표기법도 소개되었습니다. 라인을 고려
```scala
new GcdInputBundle(16).Lit(_.value1 -> x.U, _.value2 -> y.U)
```
위에서 정의한 `GcdInputBundle`에는 `value1`과 `value2`라는 두 개의 필드가 있습니다. 먼저 번들을 생성한 다음 `.Lit` 메소드를 호출하여 번들 리터럴을 생성합니다. 이 방법은 키/값 쌍의 가변 인수 목록을 사용합니다. 여기서 키(예: `_.value`)는 필드 이름이고 값(예: x.U)은 끌 하드웨어 리터럴이며, Scala `Int` x는 a로 변환됩니다. 치즐 `UInt` 리터럴. 필드 이름 앞의 `_.`는 이름 값을 번들 내부에 바인딩하는 데 필요합니다.

> 이것은 완벽한 표기법이 아닐 수도 있지만 광범위한 개발 논의에서 이것은 상용구 최소화와 Scala에서 사용할 수 있는 표기법 제한 사이의 최상의 균형으로 간주되었습니다.


In [21]:
test(new DecoupledGcd(16)) { dut =>
  dut.input.initSource().setSourceClock(dut.clock)
  dut.output.initSink().setSinkClock(dut.clock)

  val testValues = for { x <- 1 to 10; y <- 1 to 10} yield (x, y)
  val inputSeq = testValues.map { case (x, y) =>
    (new GcdInputBundle(16)).Lit(_.value1 -> x.U, _.value2 -> y.U)
  }
  val resultSeq = testValues.map { case (x, y) =>
    new GcdOutputBundle(16).Lit(_.value1 -> x.U, _.value2 -> y.U, _.gcd -> BigInt(x).gcd(BigInt(y)).U)
  }

  fork {
    dut.input.enqueueSeq(inputSeq)
  }.fork {
    for (expected <- resultSeq) {
      dut.output.expectDequeue(expected)
      dut.clock.step(5) // wait some cycles before receiving the next output to create backpressure
    }
  }.join()
}


Elaborating design...
Done elaborating.
test DecoupledGcd Success: 0 tests passed in 728 cycles in 0.107846 seconds 6750.38 Hz


---
# You're done!

[Return to the top.](#top)